# Python Insights - Analisando Dados com Python

### Case - Cancelamento de Clientes

Você foi contratado por uma empresa com mais de 800 mil clientes para um projeto de Dados. Recentemente a empresa percebeu que da sua base total de clientes, a maioria são clientes inativos, ou seja, que já cancelaram o serviço.

Precisando melhorar seus resultados ela quer conseguir entender os principais motivos desses cancelamentos e quais as ações mais eficientes para reduzir esse número.



In [2]:
import pandas as pd

# passo a passo
# paso 1: importar base de dados


tabela = pd.read_csv('data/cancelamentos.csv')
tabela = tabela.drop(columns='CustomerID')
# paso 2: visualizar a base de dados

display(tabela)



,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
881661,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
881662,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
881663,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
881664,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [5]:
# paso 3: corrigir a base de dados
tabela = tabela.dropna()
display(tabela.info())


<class 'pandas.core.frame.DataFrame'>
Index: 881659 entries, 0 to 881665
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idade                   881659 non-null  float64
 1   sexo                    881659 non-null  object 
 2   tempo_como_cliente      881659 non-null  float64
 3   frequencia_uso          881659 non-null  float64
 4   ligacoes_callcenter     881659 non-null  float64
 5   dias_atraso             881659 non-null  float64
 6   assinatura              881659 non-null  object 
 7   duracao_contrato        881659 non-null  object 
 8   total_gasto             881659 non-null  float64
 9   meses_ultima_interacao  881659 non-null  float64
 10  cancelou                881659 non-null  float64
dtypes: float64(8), object(3)
memory usage: 80.7+ MB


None

In [9]:
# paso 4: qual % de clientes cancelou
display(tabela['cancelou'].value_counts())
display(tabela['cancelou'].value_counts(normalize=True).map("{:.1%}".format))


cancelou
1.0    499993
0.0    381666
Name: count, dtype: int64

cancelou
1.0    56.7%
0.0    43.3%
Name: proportion, dtype: object

In [10]:
display(tabela['duracao_contrato'].value_counts())
display(tabela['duracao_contrato'].value_counts(normalize=True).map("{:.1%}".format))

duracao_contrato
Annual       354395
Quarterly    353059
Monthly      174205
Name: count, dtype: int64

duracao_contrato
Annual       40.2%
Quarterly    40.0%
Monthly      19.8%
Name: proportion, dtype: object

In [12]:
#agrupar tabela

agrupamento = tabela.groupby('duracao_contrato').mean(numeric_only=True)
display(agrupamento)

,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
duracao_contrato,,,,,,,,
Annual,38.842165,31.446186,15.880213,3.263401,12.465156,651.697738,14.236107,0.460760
Monthly,41.552407,30.538555,15.499274,4.985649,15.007267,550.616435,15.478012,1.000000
Quarterly,38.830938,31.419916,15.886662,3.265245,12.460863,651.427783,14.234544,0.460255


In [13]:

tabela = tabela[tabela['duracao_contrato'] != 'Monthly']
display(tabela['cancelou'].value_counts())
display(tabela['cancelou'].value_counts(normalize=True).map("{:.1%}".format))


cancelou
0.0    381666
1.0    325788
Name: count, dtype: int64

cancelou
0.0    53.9%
1.0    46.1%
Name: proportion, dtype: object

In [ ]:
#criar graficos
import plotly.express as px

# Cria o gráfico para cada coluna em tabela
for coluna in tabela.columns:
    if coluna != 'cancelou':  # Para evitar criar um histograma da coluna 'cancelou'
        grafico = px.histogram(tabela, x=coluna, color='cancelou',
                               title=f'Distribuição de {coluna} dos Usuários',
                               labels={coluna: coluna.capitalize(), 'count': 'Quantidade de Usuários'},
                               color_discrete_map={'Não': 'green', 'Sim': 'red'},
                               marginal='box')  # Adiciona um box plot à margem do histograma

        # Melhora o layout
        grafico.update_layout(
            font=dict(
                family="Arial, sans-serif",
                size=12,
                color="RebeccaPurple"
            ),
            xaxis_title_text=coluna.capitalize(),  # xaxis label with respect to subplot
            yaxis_title_text='Frequência',  # yaxis label with respect to subplot
            bargap=0.05,  # gap between bars of adjacent location coordinates
            bargroupgap=0.1  # gap between bars of the same location coordinate
        )

        # Exibe o gráfico
        grafico.show()



In [23]:
#principais causas encontradas que afetam diretamente nos cancelamentos
#pagamentos mensais
#ligações para call center acima de 4.
#dias de atrazo maior que 20, todos os clientes cancelam

tabela = tabela[tabela['ligacoes_callcenter'] < 5]
tabela = tabela[tabela['dias_atraso'] <= 20]

display(tabela['cancelou'].value_counts())
display(tabela['cancelou'].value_counts(normalize=True).map("{:.1%}".format))


cancelou
0.0    379032
1.0     85447
Name: count, dtype: int64

cancelou
0.0    81.6%
1.0    18.4%
Name: proportion, dtype: object